# basic set up

In [ ]:
#basic set up
%matplotlib inline
from netflow_main import *
import logzero
from logzero import logger
import yaml

In [16]:
def flush_log_file(log_file):
    with open(log_file, 'w'):
        pass

log_file = 'test.log'
flush_log_file(log_file)

logzero.logfile(log_file)
logger.info(f'{log_file} created.')
logger.info('------------------------------------')

def load_config(config_file, logger=logger):
    print(f"Loading config from {config_file}")
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    # print the config content to the logger file one by one
    for key in config:
        logger.info(f"{key}:")
        logger.info(f"{config[key]}:")
    logger.info('------------------------------------')
    
    return config

# TODO - make params_file an argument to the python script
config = load_config('./params/params_test.yaml' )

input_dir = config['input_dir']
output_dir = config['output_dir']
output_fig_dir = config['output_fig_dir']
pointings_file = config['pointings_file']
gurobiOptions = config['gurobiOptions']
tele_params = config['tele_params']
ra_range = config['ra_range']
dec_range = config['dec_range']
bench_config = config['bench_config']


########################
classdict = def_classdict()

# print the classdict content to the logger file one by one
for key in classdict:
    logger.info(f"{key}:")
    logger.info(f"{classdict[key]}:")
logger.info('------------------------------------')

########################
from utils.io import read_pointings
pointings = read_pointings(pointings_file)
logger.info(pointings)
logger.info('------------------------------------')




[I 250129 21:37:34 2442370712:9] test.log created.
[I 250129 21:37:34 2442370712:10] ------------------------------------
[I 250129 21:37:34 2442370712:18] input_dir:
[I 250129 21:37:35 2442370712:19] ../data_proc/test/:
[I 250129 21:37:35 2442370712:18] input_files:
[I 250129 21:37:35 2442370712:19] {'cos_file': 'cosmology_region3_3h_targets.ecsv', 'fluxstd_file': 'star_region3_3h_targets.ecsv', 'sky_file': 'sky_region3_3h_targets.ecsv'}:
[I 250129 21:37:35 2442370712:18] prefix_list:
[I 250129 21:37:35 2442370712:19] ['sci', 'cal', 'sky']:
[I 250129 21:37:35 2442370712:18] calibration_list:
[I 250129 21:37:35 2442370712:19] [False, True, True]:
[I 250129 21:37:35 2442370712:18] output_dir:
[I 250129 21:37:35 2442370712:19] ../output/test/:
[I 250129 21:37:35 2442370712:18] output_fig_dir:
[I 250129 21:37:35 2442370712:19] ../output/figures/test/:
[I 250129 21:37:35 2442370712:18] pointings_file:
[I 250129 21:37:35 2442370712:19] ../output/pointings/test/cosmology_region3_3h_pointings

Loading config from ./params/params_test.yaml


[I 250129 21:37:35 2442370712:43] sky:
[I 250129 21:37:35 2442370712:44] {'numRequired': 400, 'nonObservationCost': 1000000.0, 'calib': True}:
[I 250129 21:37:35 2442370712:43] cal:
[I 250129 21:37:35 2442370712:44] {'numRequired': 200, 'nonObservationCost': 1000000.0, 'calib': True}:
[I 250129 21:37:35 2442370712:45] ------------------------------------
[I 250129 21:37:35 2442370712:50]  ppc_code  ppc_ra ppc_dec ppc_pa ppc_resolution ppc_priority obstime
    --------- ------- ------- ------ -------------- ------------ -------
     Autumn_0 358.468   -1.27    0.0              L            1     nan
     Autumn_1 358.468  -0.075    0.0              L            1     nan
     Autumn_2 358.468    1.12    0.0              L            1     nan
     Autumn_3 358.468   2.315    0.0              L            1     nan
     Autumn_4 358.468    3.51    0.0              L            1     nan
     Autumn_5 358.468   4.706    0.0              L            1     nan
     Autumn_6 359.503  -0.672

# run netflow_main

In [ ]:
# TODO: revise the two-stage code structure
#def main(outdir, data_dir, update_tgt_file, pointings, prefix, field, Li_list, visit, fn_list, prefix_list, calibration_list, tele_params, bench, cobracoach, two_stage=False, gurobiOptions=None, collision_sim_version=2):


########################
# cosmology + sky + star targets
# put the values of the input_files into the fn_list
# put the values of the prefix into the prefix_list
# put the values of the calibration into the calibration_list
fn_list = [config['input_files'][key_name] for key_name in config['input_files'].keys()]
prefix_list = config['prefix_list']
calibration_list = config['calibration_list']

tgt, sample_sci = construct_tgt_list(catalog_path=input_dir, fn_list=fn_list, prefix_list=prefix_list, calibration_list=calibration_list)

'''
# TODO: prepare ancillary science targets (smaller size sample)
#anxillary science targets
fn_list = ["anc_select.ecsv", ]
prefix_list = ["anc", ]
calibration_list = [False, ]
        
tgt_s1, sample_sci_s1 = construct_tgt_list(catalog_path = "../data_proc/", fn_list=fn_list, prefix_list=prefix_list, calibration_list=calibration_list)

#total targets
tgt = tgt_s0 + tgt_s1
'''

n_sci = sample_sci.shape[0]
ra_sci = sample_sci[:, 1].astype('float')
dec_sci = sample_sci[:, 2].astype('float')

########################
bench, cobracoach = create_bench(layout="calibration")
collision_sim_version = 2

########################
    # one visit -- can be 1st or 2nd visit
    # TODO: set the alreadyObserved dictionary using the alreadyObserved column in the target list
    alreadyObserved={}
    for t in tgt:
        alreadyObserved[t.ID] = 0

    # set the stage of cosmology targets to be 1 (i.e. stage=0 for calibration targets to get a uniform distribution for calibration targets)
    # TODO: use CobraGroup to get a uniform distribution for calibration targets
    for t in tgt:
        if t.targetclass[:3] == 'sci': t.stage = 1

########################
#set the range for getting pointing centers
ra_min, ra_max, dec_min, dec_max = ra_range[0], ra_range[1], dec_range[0], dec_range[1]

# read the pointing centers from the file
ra_peaks, dec_peaks = read_pointings("../output/cosmology_region3_pointings.txt")

# select peaks within the range
mask_peaks = (ra_peaks>ra_min) & (ra_peaks<ra_max) & (dec_peaks>dec_min) & (dec_peaks<dec_max)
peaks = np.array([ra_peaks[mask_peaks], dec_peaks[mask_peaks]]).T
npoint = peaks.shape[0]
print("There are %d pointings." % npoint)

# save the selected pointings
with open("../output/cosmology_region3_pointings_select.txt", 'w') as f:
    f.write('# ra_pointing dec_pointing\n')
    for p in peaks:
        f.write('%.6f %.6f\n' % (p[0], p[1]))
    
# plot the pointing and samples
plt.figure(figsize=(20, 5))
print("  Your sample (gray), with the pointings (red):")
fov_diag = 1.38/2. # deg, here it's radius
mask_sci = (ra_sci>ra_min-fov_diag) & (ra_sci<ra_max+fov_diag) & (dec_sci>dec_min-fov_diag) & (dec_sci<dec_max+fov_diag)
plt.plot(ra_sci[mask_sci][::10], dec_sci[mask_sci][::10], 'k.', ms=0.2, alpha=0.5)

for raP, decP in zip(peaks[:, 0], peaks[:, 1]):
    flag_fov_reserved = plot_filter_fov(raP, decP, ra_sci[mask_sci], dec_sci[mask_sci], PA=0, n_sci_fov_least=4000, filter_fov=True)

plt.xlabel('RA', fontsize=20)
plt.ylabel('DEC', fontsize=20)
plt.savefig(outdir + 'cosmology_region3_test_fov_plot.png')
plt.show()
plt.close()

t2 = time.time()
print("Time passed - read pointings and plot for a small region: %.2f secs" % (t2-t1))
     
########################
# initialize the alreadyObserved dictionary
    alreadyObserved={}
    for t in tgt:
        alreadyObserved[t.ID] = 0
        
    # set the alreadyObserved dictionary using the alreadyObserved column in the target list (needed for the 2nd visit for sure)
    data_table = ascii.read(data_dir + prefix + 'field_{}_targets.ecsv'.format(field))
    mask_done = data_table['AlreadyObserved'] == 1
    for t in data_table['ID'][mask_done]:
        alreadyObserved[t] = 1

    # do the run
    fov_diag = 1.38/2. # >=half of the diameter of the FoV
    outfn_list = []
    tgt_id_done_list = np.array([])
    for peak, Li in zip(peaks, Li_list):
        # select the tgt list for the pointing
        tgt_ra = [tgt[i].ra for i in range(len(tgt))]
        tgt_dec = [tgt[i].dec for i in range(len(tgt))]
        # TO-DO: deal with the case when the ra is near 0 and 360
        mask_tgt = (tgt_ra>peak[0]-fov_diag) & (tgt_ra<peak[0]+fov_diag) & (tgt_dec>peak[1]-fov_diag) & (tgt_dec<peak[1]+fov_diag)
        tgt_peak = np.array(tgt)[mask_tgt].tolist()
        print("field - %s, %s (%f, %f): %d targets in the FoV." % (field, Li, peak[0], peak[1], len(tgt_peak)))

        # (Done) TODO: update the alreadyObserved dictionary
        # TODO: rewrite the write_output and cal_completeness part

        # excute the fiber assignment
        res, tpos, telescopes = execute(np.array([peak]), tgt_peak, classdict, alreadyObserved, t_obs=1800., two_stage=two_stage, collision_sim_version=collision_sim_version, bench=bench, cobracoach=cobracoach, gurobiOptions=gurobiOptions, tele_params=tele_params)

        outfn = prefix + 'field_%s_%s_visit%d.txt'%(field, Li, visit)
        write_output(tgt_peak, res, tpos, telescopes, outdir, outfn)

        outfn_list.append(outdir + outfn)
        
        # calculate the completeness and fiber assignment fraction
        comp_out_tmp1 = cal_completeness(tgt_peak, res, tpos, telescopes, Plot=True, fn_fig=outdir+prefix+'field_%s_%s_visit%d.jpg'%(field, Li, visit), \
                                        Save=True, fn_fibeff=outdir+prefix+'field_%s_%s_visit%d_fibeff.txt'%(field, Li, visit))
        
        # output the completeness and fiber assignment fraction
        data_dict = {key: [value] for key, value in comp_out_tmp1.items()}
        df = pd.DataFrame(data_dict)
        df.to_csv(outdir + prefix + 'field_%s_%s_visit%d_comp.txt'%(field, Li, visit), index=False)
        print("comp_out_tmp1 = ", comp_out_tmp1)

        # update alreadyObserved
        # TODO: revise the code for a more general case
        tgt_id_done = np.loadtxt(outdir + outfn, usecols=(0, ), unpack=True, dtype='str')
        for t in tgt_id_done:
            if(t[1:4]=='Cos'): alreadyObserved[str(t)] = 1

        tgt_id_done_list = np.append(tgt_id_done_list, tgt_id_done)

        print("%f secs passed after the visits done for field_%s_%s_visit%d." % (time.time()-t0, field, Li, visit))
        
        plot_tgt_done([outdir+outfn], outdir+'tgt_done_field_%s_%s_visit%d.jpg'%(field, Li, visit), figsize=(6, 6))

        # update the AlreadyObserved column in the target list
        # TODO: do it in a more efficient way
        if (update_tgt_file):
            mask_done = np.in1d(data_table['ID'], np.array(tgt_id_done))
            data_table['AlreadyObserved'][mask_done] = 1 
            data_table.write(data_dir + prefix + 'field_{}_targets.ecsv'.format(field), format='ascii.ecsv', overwrite=True)
            print('Update the AlreadyObserved column in ', data_dir + prefix + 'field_{}_targets.ecsv'.format(field))

                
        # update alreadyObserved
        # TODO: revise the code for a more general case
        tgt_id_done = np.loadtxt(outdir + outfn, usecols=(0, ), unpack=True, dtype='str')
        for t in tgt_id_done:
            if(t[:3]=='sci'): alreadyObserved[str(t)] = 1


# TO-DO Lists
+ fix the problem of gurobi.lic in py_env_co_fa
+ update the target list after each observation
+ second visit, pointing center shift to avoid vignetting 
+ longer exposure option for second visit
+ cobra home position and collision simulator? 
+ fiberStatus? -- bad fibers and bad Cobra
+ apply bright star mask to sky/stars
+ generate uniform sky/star fiber assignment 
>>> Masato Onodera - I don't think I can put my script to check the uniformity of the distributions of sky and fluxstd objects against the home positions of science fibers. Please see the following gist how it's made. It uses dcor package for the comparsion, so making your own should be simple.
For the GA Fornax design, there is no configuration with p<0.05 (if my calculation is right).
https://gist.github.com/monodera/4c6fef2ce6a8370338250cd78ac78f26

>>> refer GA group code -
https://github.com/Subaru-PFS-GA/ga_targeting/blob/master/nb/examples/netflow/8_cobra_groups.ipynb



# check list for the observatory Target Uploader
+ two stage == different cost (same sky coverage)?
+ CO needs to track the status of the target list and instrument
+ if different otime is used, how many targets (sci, sky, star separately) will be missed? (test myself - use output as an input)

